In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import ( Input, Dense, LSTM, BatchNormalization, LeakyReLU, Concatenate)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_excel("ASIANPAINT_Dataset.xlsx")
df = df.sort_values(["Expiry", "Date"]).reset_index(drop=True)
df["T"] = df["t"] / 365.0

In [22]:
WINDOW = 20

price_sequences = []
static_features = []
targets = []

for i in range(WINDOW, len(df)):
    
    price_sequences.append(
        df.loc[i-WINDOW:i-1, "underlying_value"].values
    )

    
    static_features.append([
        df.loc[i, "strike_price"],
        df.loc[i, "T"],
        df.loc[i, "r"],
        df.loc[i, "sigma"]
    ])

    # Target = equilibrium option price
    targets.append(df.loc[i, "close"])


In [23]:
X_seq = np.array(price_sequences)            
X_static = np.array(static_features)        
y = np.array(targets).reshape(-1, 1)


In [24]:
X_seq_train, X_seq_temp, X_static_train, X_static_temp, y_train, y_temp = train_test_split(
    X_seq, X_static, y, test_size=0.02, random_state=42
)

X_seq_val, X_seq_test, X_static_val, X_static_test, y_val, y_test = train_test_split(
    X_seq_temp, X_static_temp, y_temp, test_size=0.5, random_state=42
)


In [25]:
seq_scaler = StandardScaler()
static_scaler = StandardScaler()
y_scaler = StandardScaler()

# Scale LSTM sequences
X_seq_train = seq_scaler.fit_transform(
    X_seq_train.reshape(-1, 1)
).reshape(X_seq_train.shape)

X_seq_val = seq_scaler.transform(
    X_seq_val.reshape(-1, 1)
).reshape(X_seq_val.shape)

X_seq_test = seq_scaler.transform(
    X_seq_test.reshape(-1, 1)
).reshape(X_seq_test.shape)

# Scale static features
X_static_train = static_scaler.fit_transform(X_static_train)
X_static_val   = static_scaler.transform(X_static_val)
X_static_test  = static_scaler.transform(X_static_test)

# Scale target
y_train = y_scaler.fit_transform(y_train)
y_val   = y_scaler.transform(y_val)
y_test  = y_scaler.transform(y_test)


In [26]:

seq_input = Input(shape=(WINDOW, 1))

x = LSTM(8, return_sequences=True)(seq_input)
x = LSTM(8, return_sequences=True)(x)
x = LSTM(8)(x)


static_input = Input(shape=(X_static_train.shape[1],))


merged = Concatenate()([x, static_input])

# MLP head 
h = Dense(400)(merged)
h = LeakyReLU(alpha=0.01)(h)
h = BatchNormalization()(h)

h = Dense(400)(h)
h = LeakyReLU(alpha=0.01)(h)
h = BatchNormalization()(h)

output = Dense(1, activation="relu")(h)

model = Model(inputs=[seq_input, static_input], outputs=output)
model.summary()


C:\Users\rodes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 20, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_6 (LSTM)                 │ (None, 20, 8)             │             320 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_7 (LSTM)                 │ (None, 20, 8)             │             544 │ lstm_6[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_8 (LSTM)                 │ (None, 8)                 │             544 │ lstm_7[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_5 (InputLayer)    │ (None, 4)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 12)                │               0 │ lstm_8[0][0],              │
│                               │                           │                 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 400)               │           5,200 │ concatenate_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU)     │ (None, 400)               │               0 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 400)               │           1,600 │ leaky_re_lu_4[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 400)               │         160,400 │ batch_normalization_4[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_5 (LeakyReLU)     │ (None, 400)               │               0 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_5         │ (None, 400)               │           1,600 │ leaky_re_lu_5[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 1)                 │             401 │ batch_normalization_5[0][… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 170,609 (666.44 KB)

 Trainable params: 169,009 (660.19 KB)

 Non-trainable params: 1,600 (6.25 KB)

In [27]:
def lr_schedule(epoch):
    if epoch < 10:
        return 1e-2
    elif epoch < 20:
        return 1e-3
    else:
        return 1e-4


In [28]:
model.compile(
    optimizer=Adam(),
    loss="mse"
)

lr_callback = LearningRateScheduler(lr_schedule)

history = model.fit(
    [X_seq_train[..., np.newaxis], X_static_train],
    y_train,
    validation_data=(
        [X_seq_val[..., np.newaxis], X_static_val],
        y_val
    ),
    epochs=30,
    batch_size=4096,
    callbacks=[lr_callback],
    verbose=1
)


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 435ms/step - loss: 16.8235 - val_loss: 1.1054 - learning_rate: 0.0100
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 318ms/step - loss: 0.9388 - val_loss: 1.0985 - learning_rate: 0.0100
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 262ms/step - loss: 0.6482 - val_loss: 0.9510 - learning_rate: 0.0100
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 298ms/step - loss: 0.5404 - val_loss: 0.9533 - learning_rate: 0.0100
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - loss: 0.4881 - val_loss: 0.9311 - learning_rate: 0.0100
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 274ms/step - loss: 0.4743 - val_loss: 0.9487 - learning_rate: 0.0100
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 307ms/step - loss: 0.4662 - val_loss: 0.9147 - learning_rate: 0.0100
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 329ms/step - loss: 0.4623 - val_loss: 0.9257 - learning_rate: 0.0100
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 309ms/step - loss: 0.4562 - val_loss: 0.9206 - learning_rate: 0.0100
Epoch 10/30
9/9 ━━━━━━━━━━

In [29]:
y_pred = model.predict(
    [X_seq_test[..., np.newaxis], X_static_test]
)

y_pred = y_scaler.inverse_transform(y_pred)
y_true = y_scaler.inverse_transform(y_test)

rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
mape = np.mean(np.abs((y_pred - y_true) / y_true)) * 100

print("RMSE:", rmse)
print("MAPE (%):", mape)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
RMSE: 142.69598281290484
MAPE (%): 2936.474257338467
